In [16]:
# General
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import itertools as it

# CV
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score,train_test_split
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.model_selection import GridSearchCV, ParameterGrid
import time as time

# Models
from sklearn.linear_model import LinearRegression
from patsy import dmatrix
from pyearth import Earth
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor

In [2]:
data = pd.read_csv('./Data/OnlineNewsPopularity.csv')
data.head()

,url,timedelta,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,...,min_positive_polarity,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity,shares
0,http://mashable.com/2013/01/07/amazon-instant-...,731.0,12.0,219.0,0.663594,1.0,0.815385,4.0,2.0,1.0,...,0.100000,0.7,-0.350000,-0.600,-0.200000,0.500000,-0.187500,0.000000,0.187500,593
1,http://mashable.com/2013/01/07/ap-samsung-spon...,731.0,9.0,255.0,0.604743,1.0,0.791946,3.0,1.0,1.0,...,0.033333,0.7,-0.118750,-0.125,-0.100000,0.000000,0.000000,0.500000,0.000000,711
2,http://mashable.com/2013/01/07/apple-40-billio...,731.0,9.0,211.0,0.575130,1.0,0.663866,3.0,1.0,1.0,...,0.100000,1.0,-0.466667,-0.800,-0.133333,0.000000,0.000000,0.500000,0.000000,1500
3,http://mashable.com/2013/01/07/astronaut-notre...,731.0,9.0,531.0,0.503788,1.0,0.665635,9.0,0.0,1.0,...,0.136364,0.8,-0.369697,-0.600,-0.166667,0.000000,0.000000,0.500000,0.000000,1200
4,http://mashable.com/2013/01/07/att-u-verse-apps/,731.0,13.0,1072.0,0.415646,1.0,0.540890,19.0,19.0,20.0,...,0.033333,1.0,-0.220192,-0.500,-0.050000,0.454545,0.136364,0.045455,0.136364,505


In [3]:
data.describe()

,timedelta,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,num_videos,...,min_positive_polarity,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity,shares
count,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,...,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000
mean,354.530471,10.398749,546.514731,0.548216,0.996469,0.689175,10.883690,3.293638,4.544143,1.249874,...,0.095446,0.756728,-0.259524,-0.521944,-0.107500,0.282353,0.071425,0.341843,0.156064,3395.380184
std,214.163767,2.114037,471.107508,3.520708,5.231231,3.264816,11.332017,3.855141,8.309434,4.107855,...,0.071315,0.247786,0.127726,0.290290,0.095373,0.324247,0.265450,0.188791,0.226294,11626.950749
min,8.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,-1.000000,-1.000000,-1.000000,0.000000,-1.000000,0.000000,0.000000,1.000000
25%,164.000000,9.000000,246.000000,0.470870,1.000000,0.625739,4.000000,1.000000,1.000000,0.000000,...,0.050000,0.600000,-0.328383,-0.700000,-0.125000,0.000000,0.000000,0.166667,0.000000,946.000000
50%,339.000000,10.000000,409.000000,0.539226,1.000000,0.690476,8.000000,3.000000,1.000000,0.000000,...,0.100000,0.800000,-0.253333,-0.500000,-0.100000,0.150000,0.000000,0.500000,0.000000,1400.000000
75%,542.000000,12.000000,716.000000,0.608696,1.000000,0.754630,14.000000,4.000000,4.000000,1.000000,...,0.100000,1.000000,-0.186905,-0.300000,-0.050000,0.500000,0.150000,0.500000,0.250000,2800.000000
max,731.000000,23.000000,8474.000000,701.000000,1042.000000,650.000000,304.000000,116.000000,128.000000,91.000000,...,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.500000,1.000000,843300.000000


In [3]:
from skimpy import clean_columns
data = clean_columns(data)

60 column names have been cleaned

In [4]:
X_train, X_test, y_train, y_test = train_test_split(
    data.drop(['shares', 'url'], axis = 1),
    data['shares'],
    test_size = 0.2,
    random_state = 42)

print(X_train.shape)
print(X_test.shape)

(31715, 59)
(7929, 59)


In [5]:
train = pd.concat([X_train, y_train], axis = 1)

# Base models

## MARS

In [9]:
mars_model = Earth(max_terms = 500, max_degree = 3)
mars_model.fit(X_train,y_train)
print(mars_model.summary())

/Users/sarahhsu/opt/anaconda3/lib/python3.9/site-packages/pyearth/earth.py:813: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  pruning_passer.run()


Earth Model
--------------------------------------------------------------------------
Basis Function                                       Pruned  Coefficient  
--------------------------------------------------------------------------
(Intercept)                                          No      10385.7      
h(kw_avg_avg-6445.7)                                 No      -0.411093    
h(6445.7-kw_avg_avg)                                 No      -1.45471     
h(self_reference_avg_sharess-34808.3)                Yes     None         
h(34808.3-self_reference_avg_sharess)                No      -0.103669    
num_imgs*h(kw_avg_avg-6445.7)                        Yes     None         
h(kw_avg_min-14187.8)*num_imgs*h(kw_avg_avg-6445.7)  No      0.000112559  
h(14187.8-kw_avg_min)*num_imgs*h(kw_avg_avg-6445.7)  No      4.95434e-06  
h(kw_min_avg-666)*h(6445.7-kw_avg_avg)               Yes     None         
h(666-kw_min_avg)*h(6445.7-kw_avg_avg)               No      0.000399267  
timedelta    

/Users/sarahhsu/opt/anaconda3/lib/python3.9/site-packages/pyearth/earth.py:1066: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  coef, resid = np.linalg.lstsq(B, weighted_y[:, i])[0:2]


In [10]:
y_pred = mars_model.predict(X_test)
np.sqrt(mean_squared_error(y_test, y_pred))

10845.394434931086

## Decision Tree

In [10]:
model = DecisionTreeRegressor(random_state = 1) 
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
np.sqrt(mean_squared_error(y_test, y_pred))

17609.183620885073

In [11]:
print("Maximum tree depth =", model.get_depth())
print("Maximum leaves =", model.get_n_leaves())

Maximum tree depth = 57
Maximum leaves = 28752


## Bagged Trees

In [14]:
model2 = BaggingRegressor(estimator = DecisionTreeRegressor(), n_estimators = 150, random_state = 1,
                        n_jobs = -1)
model2.fit(X_train, y_train)

y_pred = model2.predict(X_test)
np.sqrt(mean_squared_error(y_test, y_pred))

11261.233500355822

## Random Forest

In [16]:
model3 = RandomForestRegressor(n_estimators = 150, random_state = 1, max_features = 'sqrt',
                               n_jobs = -1)
model3.fit(X_train, y_train)

y_pred = model3.predict(X_test)
np.sqrt(mean_squared_error(y_test, y_pred))

10768.44468608124

## AdaBoost

In [14]:
model4 = AdaBoostRegressor(estimator = DecisionTreeRegressor(), n_estimators = 150,
                           random_state = 1)
model4.fit(X_train, y_train)

y_pred = model4.predict(X_test)
np.sqrt(mean_squared_error(y_test, y_pred))

10954.172309167776

## Gradient Boosting

In [17]:
model5 = GradientBoostingRegressor(n_estimators = 150, random_state = 1, loss = 'huber')
model5.fit(X_train, y_train)

y_pred = model5.predict(X_test)
np.sqrt(mean_squared_error(y_test, y_pred))

10954.768297019218